# Tarea 5

**Envío del PR inicial:** 15 de octubre

**Aceptación del PR:** 21 de octubre

NOTA: Esta tarea debe entregarse en equipo.

## 1

Llamemos $c_n$ el valor del parámetro $c$ donde ocurre la bifurcación de doblamiento de periodo para el mapeo $Q_c(x)=x^2+c$, donde la órbita de periodo $2^n$ nace. Como hemos visto en notebooks anteriores, tenemos que $c_0=1/4$ marca la aparición del atractor de periodo $2^0=1$, $c_1=-1/4$ corresponde a la aparición del atractor de periodo $2^1=2$, $c_2=-3/4$ a la aparición del atractor de periodo $2^2=4$, etc. 

A partir de estos valores y otros que calcularán (al menos deben encontrar $c_6$), definimos la secuencia: $\{f_0, f_1, f_2, \dots\}$, donde

\begin{equation}
f_n = \frac{c_n-c_{n+1}}{c_{n+1}-c_{n+2}} .
\end{equation}

La pregunta es, ¿a qué valor converge esta secuencia?, es decir, dar una estimación de $f_\infty$.


*Hint:* Para realizar este ejercicio deben calcular el atractor para varios valores de $c$, de tal manera que puedan aislar las órbitas de periodo $2^p$ y de ahí determinar varios valores $c_n$. Se requerir suficiente cuidado para obtener una buena aproximación de $c_n$. 
Una opción se basa en recordar/usar que las bifurcaciones de doblamiento de periodo ocurren cuando los puntos de la órbita de periodo $p$ se tornan en repulsores, es decir, $(Q_c^p)'(x)=-1$. Esta opción, entonces, involucra obtener los valores $c_n$ a partir de órbitas periódicas de periodo $2^n$ usando los polinomios $Q_c^{2^p}(x)$ y diferenciación automática.

In [1]:
"""
    ciclosestables!(xx, f, cc, nit, nout)

Esta función itera el mapeo `f`, de una variable, `nit+nout` veces, 
usando como condición inicial `x0=0`; los últimos `nout` iterados 
actualizan al vector `xx` que tiene longitud `nout`. `cc` es el valor
del parámetro del mapeo `f`. El mapeo `f` debe ser definido de 
tal manera que `f(x0, cc)` tenga sentido. La idea es que los últimos 
`nout` iterados reflejen los ciclos estables del mapeo `f`.
"""
function ciclosestables!(xx, f, cc, nit, nout)
    @assert (nit > 0) && (nout > 0)
    
    # Primeros nit iterados
    x0 = 0.0
    for it = 1:nit
        x0 = f(x0, cc)
    end
    
    # Se guardan los siguientes nout iterados
    for it = 1:nout
        x0 = f(x0, cc)
        @inbounds xx[it] = x0
    end
    
    nothing
end

ciclosestables!

In [2]:
include("DualNum.jl")

Main.DualNum

In [3]:
using Plots

In [3]:
"""
    diag_bifurc(f, nit, nout, crange)

Itera el mapeo `f` `nit+nout` veces y regresa una matriz
cuya columna `i` tiene los últimos `nout` iterados del mapeo
para el valor del parámetro del mapeo `crange[i]`.

La función `f` debe ser definida de tal manera que `f(x0, c)` 
tenga sentido.
"""
function diag_bifurc(f, nit, nout, crange)
#     xx = Vector{Float64}(nout)
    ff = Array{Float64,2}(undef, nout, length(crange))
    
    for ic in eachindex(crange)
        c = crange[ic]
       ciclosestables!(view(ff, :, ic), f, c, nit, nout)
#         ff[:,ic] = xx
    end
    
    return ff
end

diag_bifurc

In [4]:
Qc(x,c) = x^2 + c

Qc (generic function with 1 method)

In [31]:
function C(f,c,p)
    
    A = diag_bifurc(f, 100000, p, c)
    lc = length(c)
    dc = 0
    r = 1
    g = DualNum.dual(A[1,r])
    
    while g.x´ > -1
        r = r+1
        g = DualNum.dual(A[1,r])      
        for j in 1:p
            g = f(g, c[r])
        end 
        #println(g.x´,c[r])
        if g.x´ < -1.0
            dc = c[r]
            break
        end
    end
    return dc
end

C (generic function with 1 method)

In [41]:
# sea el vector C_n
Cn = []
push!(Cn,0.25)
push!(Cn,-0.75)
push!(Cn,C(Qc,-1.24:-1/10^7:-1.26, 2))
push!(Cn,C(Qc,-1.365:-1/10^7:-1.370,4))
push!(Cn,C(Qc,-1.393:-1/10^7:-1.395,8))
push!(Cn,C(Qc,-1.395:-1/10^7:-1.4,16)) 
push!(Cn,C(Qc,-1.4:-1/10^7:-1.41,32))

7-element Array{Any,1}:
  0.25     
 -0.75     
 -1.2499734
 -1.3680878
 -1.3940417
 -1.3996295
 -1.4011925

In [38]:
fn(cn,cn1,cn2) = (cn-cn1)/(cn1-cn2) 

fn (generic function with 1 method)

In [42]:
Fn = []

for i in 1:5
    push!(Fn,fn(Cn[i],Cn[i+1],Cn[i+2]))
end

In [43]:
Fn

5-element Array{Any,1}:
 2.000106405660781
 4.232958894089118
 4.550930688644103
 4.644743906367412
 3.575047984644977

In [29]:
R = diag_bifurc(Qc, 10000, 32, -1.4015:-1/10^7:-1.42)

32×185001 Array{Float64,2}:
 -1.40091    -1.40051    -1.40003    …  -1.37418     -1.33793    -1.41698  
  0.56104     0.559918    0.558585       0.468363     0.370044    0.587827 
 -1.08673    -1.08799    -1.08948       -1.20064     -1.28307    -1.07446  
 -0.220508   -0.217774   -0.214526       0.0215263    0.226263   -0.265536 
 -1.35288    -1.35407    -1.35548       -1.41954     -1.36881    -1.34949  
  0.428774    0.432018    0.435823   …   0.595084     0.453627    0.401125 
 -1.21765    -1.21486    -1.21156       -1.06588     -1.21422    -1.2591   
  0.0811787   0.0743863   0.0663743     -0.28391      0.0543355   0.16533  
 -1.39491    -1.39597    -1.39709       -1.33939     -1.41705    -1.39267  
  0.544274    0.547223    0.550373       0.373979     0.588024    0.519519 
 -1.10527    -1.10205    -1.09859    …  -1.28014     -1.07423    -1.1501   
 -0.179887   -0.186993   -0.194601       0.218758    -0.266035   -0.0972699
 -1.36914    -1.36653    -1.36363       -1.37214     -1.3492

In [30]:
ccc =  -1.4015:-1/10^7:-1.42
for i in 1:50
    d = DualNum.dual(R[1,i])
    for j in 1:32
        d = Qc(d, ccc[i])
    end
    println(d, ccc[i])
end

-1.4005680194862877 + 3.0817404860511175 ϵ
-1.4015
-1.3999392669935642 -0.35954367735046555 ϵ
-1.4015001
-1.4009020329825488 -2.6902487683081264 ϵ
-1.4015002
-1.4011079199736445 -4.6416521304159755 ϵ
-1.4015003
-1.4009003563658577 + 3.1970853777229973 ϵ
-1.4015004
-1.4008995812779288 + 3.19768325944695 ϵ
-1.4015005
-1.4008992035292842 + 3.1980752776708425 ϵ
-1.4015006
-1.4008989298692964 + 3.198413421481101 ϵ
-1.4015007
-1.4008987116199147 + 3.1987229111747166 ϵ
-1.4015008
-1.4008985292280576 + 3.199013874340542 ϵ
-1.4015009
-1.4008983724938742 + 3.199291592555149 ϵ
-1.401501
-1.4008982352964179 + 3.199559234425215 ϵ
-1.4015011
-1.4008981136202567 + 3.19981887803994 ϵ
-1.4015012
-1.400898004662512 + 3.2000719735663643 ϵ
-1.4015013
-1.4008979063747806 + 3.2003195805258637 ϵ
-1.4015014
-1.4008978172061264 + 3.200562500916345 ϵ
-1.4015015
-1.4008977359529442 + 3.200801357038255 ϵ
-1.4015016
-1.400897668057854 + 3.201033466344521 ϵ
-1.4015017
-1.400898287352128 + 3.200923478822518 ϵ
-1.401

## 2

Repitan el ejercicio anterior para el mapeo $S_c(x) = c \sin(x)$. 

- ¿Cómo se comparan los valores obtenidos de $f_n$? 

- ¿Qué interpretación le pueden dar a este resultado, en comparación del ejercicio anterior?

## 3

Como se ve en la Fig. 1 del diagrama de bifurcaciones de $Q_c$, $x=0$ pertenece a un ciclo de periodo $2^n$ para ciertos valores $C_n$ del parámetro. Dichos valores son *especiales*, ya que el hecho de que $x=0$ pertenezca a un ciclo de periodo $2^n$ define los llamados *ciclos superestable*, donde tenemos $(Q^{2^p}_{C_n})'(0)=0$.

- ¿A qué converge la secuencia $f_n$, definida ahora a partir de los valores $C_n$.

- De los $2^p$ puntos del ciclo de periodo $2^p$, es decir, $\{0, p_1, \dots p_{2^{n-1}}\,\}$ hay uno (distinto del 0) cuya distancia a 0 es la menor; a esa distancia la identificaremos como $d_n$. Calcular numéricamente a qué converge la secuencia $d_n/d_{n+1}$.